In [1]:
import urbans
from dictionary import dic_en2jp

In [25]:
"""
TODO: Terminals for participles
V VT VsrT VMod VsrMod VPassOrgT VPassOrgMod

""" 
src_grammar = """S -> NPQ VP 
S -> NPQ was Nominal 
S -> NPQ did NPVMod 
S -> was NominalSub Vobl 
S -> NPQ Vobl 
S -> was NominalSub Adj 
S -> was NominalSub Nominal 
S -> did NominalSub VP 

NPVMod -> Nominal VMod
NPVMod -> Nominal VPrepOrgMod
NPVT -> Nominal VT
NPVT -> Nominal VPrepOrgT

VP -> V Nominal
VP -> was Vobl
VPT -> VT Nominal
VPT -> was VoblT 
VPMod -> VMod Nominal
VPMod -> was VoblMod

VPrep -> was VPrep
VPrep -> Vsr by
VPrepT -> was VPrepT
VPrepT -> VsrT by
VPrepMod -> was VPrepMod
VPrepMod -> VsrMod by

VPrepOrgMod -> was VPrepOrgMod
VPrepOrgMod -> VPassOrgMod by
VPrepOrgT -> was VPrepOrgT
VPrepOrgT -> VPassOrgT by

Vobl -> VPrep Nominal
VoblT -> VPrepT Nominal 
VoblMod -> VPrepMod Nominal

NPQ -> WhW Nominal 
NPQ -> WhW role caseO

commonNoun -> commonNoun RC

RC -> VoblMod
RC -> R VPMod
RC -> R NPVMod
RC -> whose role VPMod

VP -> VPT andVP
VP -> VPx andVP
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVP -> conj VP 
andVP -> punct conj VP
punctVPT -> punct VPT

VPMod -> VPT andVPMod
VPMod -> VPx andVPMod
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVPMod -> conj VPMod
andVPMod -> punct conj VPMod
punctVPT -> punct VPT

Vobl -> VoblT andVobl
Vobl -> Voblx andVobl 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVobl -> conj Vobl 
andVobl -> punct conj Vobl
punctVoblT -> punct VoblT

VoblMod -> VoblT andVoblMod
VoblMod -> Voblx andVoblMod 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVoblMod -> conj VoblMod 
andVoblMod -> punct conj VoblMod
punctVoblT -> punct VoblT

VPrep -> VPrepT andVPrep 
VPrep -> VPrepX andVPrep 
VPrepX -> VPrepT punctVPrepT 
VPrepX -> VPrepX punctVPrepT 
andVPrep -> conj VPrep 
andVPrep -> punct conj VPrep 
punctVPrepT -> punct VPrepT

VPrepMod -> VPrepT andVPrepMod 
VPrepMod -> VPrepX andVPrepMod 
VPrepX -> VPrepT punctVPrepT 
VPrepX -> VPrepX punctVPrepT 
andVPrepMod -> conj VPrepMod 
andVPrepMod -> punct conj VPrepMod 
punctVPrepT -> punct VPrepT

VPrepOrgMod -> VPrepOrgT andVPrepOrgMod 
VPrepOrgMod -> VPrepOrgX andVPrepOrgMod 
VPrepOrgX -> VPrepOrgT punctVPrepOrgT 
VPrepOrgX -> VPrepOrgX punctVPrepOrgT 
andVPrepOrgMod -> conj VPrepOrgMod 
andVPrepOrgMod -> punct conj VPrepOrgMod 
punctVPrepOrgT -> punct VPrepOrgT

V -> VT andV
V -> Vx andV
Vx -> VT punctVT
Vx -> Vx punctVT 
andV -> conj V
andV -> punct conj V 
punctVT -> punct VT

VT -> VT andVT
VT -> Vx andVT
Vx -> VT punctVT
Vx -> Vx punctVT 
andVT -> conj VT
andVT -> punct conj VT 
punctVT -> punct VT

VMod -> VT andVMod
VMod -> Vx andVMod
andVMod -> conj VMod
andVMod -> punct conj VMod

Vsr -> VsrT andVsr
Vsr -> Vsrx andVsr
Vsrx -> VsrT punctVsrT
Vsrx -> Vsrx punctVsrT 
andVsr -> conj Vsr
andVsr -> punct conj Vsr 
punctVsrT -> punct VsrT

VsrT -> VsrT andVsrT
VsrT -> Vsrx andVsrT
andVsrT -> conj VsrT
andVsrT -> punct conj VsrT 
punctVsrT -> punct VsrT

VsrMod -> VsrT andVsrMod
VsrMod -> Vsrx andVsrMod
andVsrMod -> conj VsrMod
andVsrMod -> punct conj VsrMod

Vx -> Vx punctVPrepT
Vx -> VT punctVPrepT
V -> Vx andVPrep
V -> VT andVPrep
VPrep -> VPrepT andV
VPrep -> VPrepX andV 
VPrepX -> VPrepT punctVT 
VPrepX -> VPrepX punctVT


VPrepOrgMod -> VPrepOrgT andVOrgPassMod
VPrepOrgMod -> VPrepOrgX andVOrgPassMod
VPrepOrgX -> VPrepOrgT punctVOrgPassT
VPrepOrgX -> VPrepOrgX punctVOrgPassT

punctVOrgPassT -> punct VOrgPassT
andVOrgPassMod -> conj VOrgPassMod
andVOrgPassMod -> punct conj VOrgPassMod


VPassOrgMod -> VPassOrgT andVPassOrgMod
VPassOrgMod -> VPassOrgx andVPassOrgMod
VPassOrgx -> VPassOrgT punctVPassOrgT
VPassOrgx -> VPassOrgx punctVPassOrgT 
andVPassOrgMod -> conj VPassOrgMod
andVPassOrgMod -> punct conj VPassOrgMod 
punctVPassOrgT -> punct VPassOrgT


VMod -> Vx andVPrepMod
VMod -> VT andVPrepMod
VPrepMod -> VPrepT andVMod
VPrepMod -> VPrepX andVMod

NPVMod -> NPVT andNPVMod
NPVMod -> NPVx andNPVMod 
NPVx -> NPVT punctNPVT
NPVx -> NPVx punctNPVT 
andNPVMod -> conj NPVMod 
andNPVMod -> punct conj NPVMod 
punctNPVT -> punct NPVT

V -> F V
VT -> F VT
VMod -> F VMod
Vsr -> F Vsr
VsrT -> F VsrT
VsrMod -> F VsrMod
VPassOrgMod -> F VPassOrgMod
VPassOrgT -> F VPassOrgT
VOrgPassT -> F VOrgPassT
VOrgPassMod -> F VOrgPassMod


Nominal -> Name 
Nominal -> DP
Nominal -> commonNoun

NominalSub -> Name
NominalSub -> DPSub
NominalSub -> commonNoun

DP -> caseS role
DPs -> caseS role
DPSub -> caseSSub role
DPSubs -> caseSSub role

caseS -> DPs pS 
caseS -> Name pS 
caseSSub -> DPSubs pS
caseSSub -> Name pS

DP -> det role caseO 
DPSub -> det role caseOSub

caseO -> of DP 
caseO -> of Name
caseOSub -> of DPSub
caseOSub -> of Name

DP -> det commonNoun
DPs -> det commonNoun
DPSub -> detSub commonNoun
DPSubs -> detSub commonNoun

Name -> Name andName 
Name -> Namex andName 
Namex -> Name punctName 
Namex -> Namex punctName 
andName -> conjN Name 
andName -> punctN conjN Name 
punctName -> punctN Name

commonNoun -> commonNoun andCommonNoun
commonNoun -> commonNounx andCommonNoun
commonNounx -> commonNoun punctCommonNoun
commonNounx -> commonNounx punctCommonNoun
andCommonNoun -> conjN commonNoun 
andCommonNoun -> punctN conjN commonNoun 
punctCommonNoun -> punctN commonNoun

role -> role androle
role -> rolex androle 
rolex -> role punctrole 
rolex -> rolex punctrole 
androle -> conjN role 
androle -> punctN conjN role
punctrole -> punctN role

commonNoun -> F commonNounHead 
commonNounHead -> F commonNounHead 
role -> F roleHead
role -> Cnt of nat
commonNoun -> P commonNounHead
commonNounHead -> P commonNounHead

commonNoun -> AdjNa commonNounHead 
role -> AdjNa roleHead







punct -> ","
punctN -> ","

Cnt -> 'country'
nat -> 'nationality'
P -> 'production'
F -> 'film' | 'art' | 'executive' | 'costume'

V -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
Vsr -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VsrT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VsrMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VPassOrgT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VPassOrgMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VOrgPassT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VOrgPassMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'




Name -> 'M0' | 'M1' | 'M2' | 'M3' | 'M4' | 'M5' | 'M6' | 'M7' | 'M8' | 'M9'

commonNoun -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'
commonNounHead -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'

role -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'
roleHead -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'


NPQ -> 'who' | 'what' | 'Who' | 'What'
WhW -> 'What' | 'Which' | 'what' | 'which'
did -> 'did' | 'Did'

conj -> 'and'
conjN -> 'and'

pS -> "'s"
of -> 'of'

det -> 'a' | 'an'
detSub -> 'a' | 'an'

by -> 'by'
Adj -> 'female' | 'American' | 'French' | 'Italian' | 'male' | 'Swedish' | 'Canadian' | 'British' | 'Spanish' | 'Mexican' | 'Chinese' | 'German' | 'American' | 'Dutch' | 'Japanese'
AdjNa -> 'female' | 'American' | 'French' | 'Italian' | 'male' | 'Swedish' | 'Canadian' | 'British' | 'Spanish' | 'Mexican' | 'Chinese' | 'German' | 'American' | 'Dutch' | 'Japanese'
was -> 'was' | 'were' | 'Was' | 'Were'
R -> 'that'
whose -> 'whose'
Qmark -> '?'






"""

In [26]:
src_to_target_grammar = {
    "S -> NPQ VP":"S -> NPQ が VP か", 
    "S -> NPQ was Nominal":"S -> Nominal は NPQ でしたか", # Q
    "S -> NPQ did NPVMod":"S -> NPVMod のは NPQ でしたか", # what did A write and B edit? -> A ga kaite, B ga hennshushita no ha nann deshitaka
    "S -> was NominalSub Vobl":"S -> NominalSub は Vobl か",
    "S -> NPQ Vobl":"S -> NPQ が Vobl か", # same Q
    "S -> was NominalSub Adj":"S -> NominalSub は Adj でしたか",
    "S -> was NominalSub Nominal":"S -> NominalSub は Nominal でしたか",
    "S -> did NominalSub VP":"S -> NominalSub は VP か",

    # TODO: Chain rules to add for VMod and VPrepOrgMod
    "NPVMod -> Nominal VMod":"NPVMod -> Nominal が VMod",
    # todo: this rule basically follows VP, VPrepOrg finally derives as V
    "NPVMod -> Nominal VPrepOrgMod":"NPVMod -> Nominal を VPrepOrgMod", # VPrepOrg: passive in EN, active in JP 
    "NPVT -> Nominal VT":"NPVT -> Nominal が VT",
    "NPVT -> Nominal VPrepOrgT":"NPVT -> Nominal を VPrepOrgT",

    "VP -> V Nominal":"VP -> Nominal を V", # have lunch -> hirugohan wo taberu
    "VP -> was Vobl":"VP -> Vobl", # was done by him -> kare ni sareta
    "VPT -> VT Nominal":"VPT -> Nominal を VT",
    "VPT -> was VoblT":"VPT -> VoblT",
    "VPMod -> VMod Nominal":"VPMod -> Nominal を VMod",
    "VPMod -> was VoblMod":"VPMod -> VoblMod",

    "VPrep -> was VPrep":"VPrep -> VPrep",
    "VPrep -> Vsr by":"VPrep -> Vsr",
    "VPrepT -> was VPrepT":"VPrepT -> VPrepT",
    "VPrepT -> VsrT by":"VPrepT -> VsrT",
    "VPrepMod -> was VPrepMod":"VPrepMod -> VPrepMod",
    "VPrepMod -> VsrMod by":"VPrepMod -> VsrMod",
 
    "VPrepOrgMod -> was VPrepOrgMod":"VPrepOrgMod -> VPrepOrgMod",
    "VPrepOrgMod -> VPassOrgMod by":"VPrepOrgMod -> VPassOrgMod",
    "VPrepOrgT -> was VPrepOrgT":"VPrepOrgT -> VPrepOrgT",
    "VPrepOrgT -> VPassOrgT by":"VPrepOrgT -> VPassOrgT",

    "Vobl -> VPrep Nominal":"Vobl -> Nominal に VPrep", # done by nominal -> nominal ni sareta
    "VoblT -> VPrepT Nominal":"VoblT -> Nominal に VPrepT",
    "VoblMod -> VPrepMod Nominal":"VoblMod -> Nominal に VPrepMod",
    
    # "NPQ -> WhW Nominal":"NPQ -> WhW Nominal", # which city -> dono machi
    # TODO: caseO -> of ... : caseO ... no
    "NPQ -> WhW role caseO":"NPQ -> caseO WhW role", # which character of Alice -> alisu no dono kyarakuta

    "commonNoun -> commonNoun RC":"commonNoun -> RC commonNoun", # people whose role was... -> yaku ga... hito

    # "RC -> VoblMod":"RC -> VoblMod", # people attacked by... -> ...ni kougekisareta hito 
    "RC -> R VPMod":"RC -> VPMod", # people that eat... -> ...wo taberu hito
    "RC -> R NPVMod":"RC -> NPVMod", # people that you like -> anata ga sukina hito
    "RC -> whose role VPMod":"RC -> role が VPMod", # todo: role GA -> ...

    # "VP -> VPT andVP":"VP -> VPT andVP",
    # "VP -> VPx andVP":"VP -> VPx andVP",
    # "VPx -> VPT punctVPT":"VPx -> VPT punctVPT",
    # "VPx -> VPx punctVPT":"VPx -> VPx punctVPT",
    # "andVP -> conj VP":"andVP -> conj VP",
    # "andVP -> punct conj VP":"andVP -> punct conj VP",
    # "punctVPT -> punct VPT":"punctVPT -> punct VPT",

    # VPMod -> VPT andVPMod
    # VPMod -> VPx andVPMod
    # VPx -> VPT punctVPT 
    # VPx -> VPx punctVPT 
    # andVPMod -> conj VPMod
    # andVPMod -> punct conj VPMod
    # punctVPT -> punct VPT

    # Vobl -> VoblT andVobl
    # Vobl -> Voblx andVobl 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVobl -> conj Vobl 
    # andVobl -> punct conj Vobl
    # punctVoblT -> punct VoblT

    # VoblMod -> VoblT andVoblMod
    # VoblMod -> Voblx andVoblMod 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVoblMod -> conj VoblMod 
    # andVoblMod -> punct conj VoblMod
    # punctVoblT -> punct VoblT

    # VPrep -> VPrepT andVPrep 
    # VPrep -> VPrepX andVPrep 
    # VPrepX -> VPrepT punctVPrepT 
    # VPrepX -> VPrepX punctVPrepT 
    # andVPrep -> conj VPrep 
    # andVPrep -> punct conj VPrep 
    # punctVPrepT -> punct VPrepT

    # VPrepMod -> VPrepT andVPrepMod 
    # VPrepMod -> VPrepX andVPrepMod 
    # VPrepX -> VPrepT punctVPrepT 
    # VPrepX -> VPrepX punctVPrepT 
    # andVPrepMod -> conj VPrepMod 
    # andVPrepMod -> punct conj VPrepMod 
    # punctVPrepT -> punct VPrepT

    # VPrepOrgMod -> VPrepOrgT andVPrepOrgMod 
    # VPrepOrgMod -> VPrepOrgX andVPrepOrgMod 
    # VPrepOrgX -> VPrepOrgT punctVPrepOrgT 
    # VPrepOrgX -> VPrepOrgX punctVPrepOrgT 
    # andVPrepOrgMod -> conj VPrepOrgMod 
    # andVPrepOrgMod -> punct conj VPrepOrgMod 
    # punctVPrepOrgT -> punct VPrepOrgT

    # V -> VT andV
    # V -> Vx andV
    # Vx -> VT punctVT
    # Vx -> Vx punctVT 
    # andV -> conj V
    # andV -> punct conj V 
    # punctVT -> punct VT

    # VT -> VT andVT
    # VT -> Vx andVT
    # Vx -> VT punctVT
    # Vx -> Vx punctVT 
    # andVT -> conj VT
    # andVT -> punct conj VT 
    # punctVT -> punct VT

    # VMod -> VT andVMod
    # VMod -> Vx andVMod
    # andVMod -> conj VMod
    # andVMod -> punct conj VMod

    # Vsr -> VsrT andVsr
    # Vsr -> Vsrx andVsr
    # Vsrx -> VsrT punctVsrT
    # Vsrx -> Vsrx punctVsrT 
    # andVsr -> conj Vsr
    # andVsr -> punct conj Vsr 
    # punctVsrT -> punct VsrT

    # VsrT -> VsrT andVsrT
    # VsrT -> Vsrx andVsrT
    # andVsrT -> conj VsrT
    # andVsrT -> punct conj VsrT 
    # punctVsrT -> punct VsrT

    # VsrMod -> VsrT andVsrMod
    # VsrMod -> Vsrx andVsrMod
    # andVsrMod -> conj VsrMod
    # andVsrMod -> punct conj VsrMod

    # Vx -> Vx punctVPrepT
    # Vx -> VT punctVPrepT
    # V -> Vx andVPrep
    # V -> VT andVPrep
    # VPrep -> VPrepT andV
    # VPrep -> VPrepX andV 
    # VPrepX -> VPrepT punctVT 
    # VPrepX -> VPrepX punctVT


    # VPrepOrgMod -> VPrepOrgT andVOrgPassMod
    # VPrepOrgMod -> VPrepOrgX andVOrgPassMod
    # VPrepOrgX -> VPrepOrgT punctVOrgPassT
    # VPrepOrgX -> VPrepOrgX punctVOrgPassT

    # punctVOrgPassT -> punct VOrgPassT
    # andVOrgPassMod -> conj VOrgPassMod
    # andVOrgPassMod -> punct conj VOrgPassMod


    # VPassOrgMod -> VPassOrgT andVPassOrgMod
    # VPassOrgMod -> VPassOrgx andVPassOrgMod
    # VPassOrgx -> VPassOrgT punctVPassOrgT
    # VPassOrgx -> VPassOrgx punctVPassOrgT 
    # andVPassOrgMod -> conj VPassOrgMod
    # andVPassOrgMod -> punct conj VPassOrgMod 
    # punctVPassOrgT -> punct VPassOrgT


    # VMod -> Vx andVPrepMod
    # VMod -> VT andVPrepMod
    # VPrepMod -> VPrepT andVMod
    # VPrepMod -> VPrepX andVMod

    # NPVMod -> NPVT andNPVMod
    # NPVMod -> NPVx andNPVMod 
    # NPVx -> NPVT punctNPVT
    # NPVx -> NPVx punctNPVT 
    # andNPVMod -> conj NPVMod 
    # andNPVMod -> punct conj NPVMod 
    # punctNPVT -> punct NPVT

    # V -> F V
    # VT -> F VT
    # VMod -> F VMod
    # Vsr -> F Vsr
    # VsrT -> F VsrT
    # VsrMod -> F VsrMod
    # VPassOrgMod -> F VPassOrgMod
    # VPassOrgT -> F VPassOrgT
    # VOrgPassT -> F VOrgPassT
    # VOrgPassMod -> F VOrgPassMod


    # Nominal -> Name 
    # Nominal -> DP
    # Nominal -> commonNoun

    # NominalSub -> Name
    # NominalSub -> DPSub
    # NominalSub -> commonNoun


    "DP -> det role caseO":"DP -> caseO role",
    "DPSub -> det role caseOSub":"DPSub -> caseOSub role",
# of -> no
    "caseO -> of DP":"caseO -> DP of",
    "caseO -> of Name":"caseO -> Name of",
    "caseOSub -> of DPSub":"caseOSub -> DPSub of",
    "caseOSub -> of Name":"caseOSub -> Name of",

    # DP -> det commonNoun
    # DPSub -> detSub commonNoun

    # Name -> Name andName 
    # Name -> Namex andName 
    # Namex -> Name punctName 
    # Namex -> Namex punctName 
    # andName -> conjN Name 
    # andName -> punctN conjN Name 
    # punctName -> punctN Name

    # commonNoun -> commonNoun andCommonNoun
    # commonNoun -> commonNounx andCommonNoun
    # commonNounx -> commonNoun punctCommonNoun
    # commonNounx -> commonNounx punctCommonNoun
    # andCommonNoun -> conjN commonNoun 
    # andCommonNoun -> punctN conjN commonNoun 
    # punctCommonNoun -> punctN commonNoun

    # role -> role androle
    # role -> rolex androle 
    # rolex -> role punctrole 
    # rolex -> rolex punctrole 
    # androle -> conjN role 
    # androle -> punctN conjN role
    # punctrole -> punctN role

    # commonNoun -> F commonNoun 
    # role -> F role
    # role -> Cnt of nat
    # commonNoun -> P commonNoun

    "role -> Cnt of nat":"role -> nat Cnt" # country of nationality -> 国籍国
    
    # commonNoun -> AdjNa commonNoun 
    # role -> AdjNa role
}

special cases: like -> ga suki

In [27]:
word_parser = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dict())

In [5]:
with open('Questions.txt') as f:
    lines = f.readlines()


In [6]:
lines = list(set(lines))

In [7]:
del lines[lines.index('\n')]

## Word Parsing

In [8]:
tag_word_set, failed_sentences, ambiguity_sentences = word_parser.parse_words(sentences=lines)

100%|██████████| 105461/105461 [16:25<00:00, 106.98it/s]

Word parsing completed! 7004 sentences failed. 19823 sentences occurred ambiguity.


In [6]:
# reparsing
tag_word_set_2, failed_sentences_2, ambiguity_sentences_2 = word_parser.parse_words(sentences=['Was a Chinese actor that M2 was written by and starred a composer\n'])

Word parsing completed! 0 sentences failed. 1 sentences occurred ambiguity.


In [15]:
tag_word_set.keys()

dict_keys(['did', 'Name', 'V', 'det', 'role', 'of', 'commonNoun', 'pS', 'AdjNa', 'F', 'VT', 'conjN', 'conj', 'punct', 'R', 'VMod', 'P', 'punctN', 'VsrT', 'by', 'VsrMod', 'was', 'whose', 'VPassOrgMod', 'Vsr', 'VPassOrgT', 'VOrgPassMod', 'Cnt', 'nat', 'VOrgPassT', 'detSub', 'Adj', 'WhW', 'NPQ'])

In [32]:
tag_word_set['NPQ']

{'What', 'Who'}

In [13]:
# ambiguities caused by patterns like V -> F V, Noun -> P Noun 
F = ['film', 'art', 'executive', 'costume', 'production']

In [14]:
filter_amb = [x for x in list(ambiguity_sentences.keys()) if
              all(y  not in x for y in F)]
print(len(filter_amb))
filter_amb

10587


["Did M2 star M0 's editor and star and star M1 's employee\n",
 "Did M1 edit and write a sequel of M0 's sequel\n",
 "Did M2 and M4 star a Japanese writer of M1 's prequel and star M3\n",
 'Did M2 star M3 and M4 and star a Mexican employee and founder of M0\n',
 "Did M1 and M2 found a employer of M0 's star\n",
 "Did M1 influence a sibling of M0 's founder\n",
 "Did M1 and M2 influence a founder and employee of M0 's producer and distributor\n",
 'Did M3 and M6 star a Chinese composer that M2 starred and star M4 and M5\n',
 "Did M1 produce a prequel of M0 's sequel\n",
 "Did M1 star a cinematographer , director , and writer of M0 's sequel\n",
 "Did M3 and M4 marry a Swedish spouse of M1 's American sibling\n",
 'Did M3 and M4 marry a French cinematographer that M1 employed\n',
 'Did M3 star M4 and star a male person that M2 married\n',
 'Did M2 and M3 star a female founder and employee of M0\n',
 'Did M2 marry a American writer and cinematographer of M0\n',
 "Did M2 influence M3 and 

In [205]:
import nltk
analyze_sen = 'Was a Chinese actor that M2 was written by and starred a composer\n'
for i in ambiguity_sentences[analyze_sen]:
    i.draw()
ambiguity_sentences[analyze_sen]

[Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('DPSub', [Tree('detSub', ['a']), Tree('commonNoun', [Tree('commonNoun', [Tree('AdjNa', ['Chinese']), Tree('commonNoun', ['actor'])]), Tree('RC', [Tree('R', ['that']), Tree('NPVMod', [Tree('Nominal', [Tree('Name', ['M2'])]), Tree('VPrepOrgMod', [Tree('VPrepOrgT', [Tree('was', ['was']), Tree('VPrepOrgT', [Tree('VPassOrgT', ['written']), Tree('by', ['by'])])]), Tree('andVOrgPassMod', [Tree('conj', ['and']), Tree('VOrgPassMod', ['starred'])])])])])])])]), Tree('Nominal', [Tree('DP', [Tree('det', ['a']), Tree('commonNoun', ['composer'])])])]),
 Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('DPSub', [Tree('detSub', ['a']), Tree('commonNoun', [Tree('commonNoun', [Tree('AdjNa', ['Chinese']), Tree('commonNoun', ['actor'])]), Tree('RC', [Tree('R', ['that']), Tree('NPVMod', [Tree('Nominal', [Tree('Name', ['M2'])]), Tree('VPrepOrgMod', [Tree('was', ['was']), Tree('VPrepOrgMod', [Tree('VPrepOrgT', [Tree('VPassOrgT', ['written'])

In [152]:
ambiguity_sentences_2['Was M2 produced by a production company that distributed M1\n']

[Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('Name', ['M2'])]), Tree('Vobl', [Tree('VPrep', [Tree('Vsr', ['produced']), Tree('by', ['by'])]), Tree('Nominal', [Tree('DP', [Tree('det', ['a']), Tree('commonNoun', [Tree('commonNoun', [Tree('P', ['production']), Tree('commonNoun', ['company'])]), Tree('RC', [Tree('R', ['that']), Tree('VPMod', [Tree('VMod', ['distributed']), Tree('Nominal', [Tree('Name', ['M1'])])])])])])])])]),
 Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('Name', ['M2'])]), Tree('Vobl', [Tree('VPrep', [Tree('Vsr', ['produced']), Tree('by', ['by'])]), Tree('Nominal', [Tree('DP', [Tree('det', ['a']), Tree('commonNoun', [Tree('P', ['production']), Tree('commonNoun', [Tree('commonNoun', ['company']), Tree('RC', [Tree('R', ['that']), Tree('VPMod', [Tree('VMod', ['distributed']), Tree('Nominal', [Tree('Name', ['M1'])])])])])])])])])])]

In [187]:
failed_sentences

['\n', '\n', '\n']

## Translation

In [30]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)

In [31]:
trans_sentence, trans_map = e2j_translator.translate(ambiguity_sentences, remove_space=True)

100%|██████████| 19823/19823 [05:08<00:00, 64.32it/s]


In [36]:
len(trans_map)

784

In [37]:
trans_map

[{'M3とM5はM1に影響されてそしてM2に影響されてそしてM4に編集された撮影監督に監督されましたか': 10,
  'M3とM5はM1に影響されてそしてM2に影響された撮影監督に監督されてそしてM4に編集されましたか': 10,
  'M3とM5はM1に影響された撮影監督に監督されてそしてM2に影響されてそしてM4に編集されましたか': 10},
 {'あるスウェーデンの映画はM1に雇用されたプロデューサでしたか': 4, 'あるスウェーデンの映画プロデューサはM1に雇用されましたか': 3},
 {'M1とM2が結婚したのはどの雇用主がM3の作家と撮影監督を雇用した映画プロデューサでしたか': 5,
  'M1とM2が結婚したのはどの雇用主がM3の作家を雇用した映画プロデューサと撮影監督でしたか': 5},
 {'ある映画はM1、M2、M3、とM4に影響された編集者でしたか': 4, 'ある映画編集者はM1、M2、M3、とM4に影響されましたか': 3},
 {'ある映画はM3の前日譚の監督に影響されてそしてM1とM2に影響されたプロデューサでしたか': 6,
  'ある映画プロデューサはM3の前日譚の監督に影響されてそしてM1とM2に影響されましたか': 5},
 {'ある映画はM1、M2、M3、M4、M5、とM6に雇用されてそしてM7に雇用された監督でしたか': 6,
  'ある映画監督はM1、M2、M3、M4、M5、とM6に雇用されてそしてM7に雇用されましたか': 5},
 {'ある映画はM1に雇用された監督でしたか': 4, 'ある映画監督はM1に雇用されましたか': 3},
 {'ある映画はM1、M2、とM3に雇用されてそしてM4、M5、とM6に雇用された監督でしたか': 6,
  'ある映画監督はM1、M2、とM3に雇用されてそしてM4、M5、とM6に雇用されましたか': 5},
 {'何がM2を書いたスウェーデンの人にプロデュースされてそしてM0を主演させてそして監督されましたか': 11,
  '何がM2を書いてそして主演したスウェーデンの人にプロデュースされてそしてM0に監督されましたか': 11,
  '何がM2を書いたスウェーデンの人にプロデュースされてそしてM0に主演してそして監督されましたか': 11},
 {'何がM5の創設者を

#### Disambiguity
784 out of 19823 ambiguities harmful

In [ ]:
for tm in trans_map:
    

In [40]:
trans_sentence

['M2が買収したのはどのM1が買収した制作会社でしたか']

In [11]:
trans_sentence, trans_map = e2j_translator.translate(lines, remove_space=True)

100%|██████████| 105461/105461 [18:03<00:00, 97.30it/s] 


In [38]:
lines[7]

'What production company that M1 acquired did M2 acquire\n'

In [35]:
trans_sentence

['どの映画がM1に書かれて、エグゼクティブプロデュースされて、監督されて、そして編集されましたか',
 '何がM0の編集者と主演に創設されましたか',
 '何がM0の創設者に編集されてそして作曲家に編集されましたか',
 '何がM0の監督、編集者、主演、と作曲家に監督されてそしてプロデュースされましたか',
 '何がM1、M2、とM3を監督した人に編集されて、書かれて、そしてプロデュースされましたか',
 '何がM0に配給されて、M3の日本の撮影監督にプロデュースされて、そしてM1にプロデュースされましたか',
 'どの映画がM2の配偶者に監督されてそして衣裳デザイナーに編集されましたか',
 'M2とM3は映画プロデューサに編集されてそして会社の被雇用者の兄弟姉妹に監督されましたか',
 'M0、M1、とM2の主演がエグゼクティブプロデュースして、プロデュースして、そして監督したのは何でしたか',
 'M0の兄弟姉妹がエグゼクティブプロデュースして、監督して、書いて、そして編集したのは何でしたか',
 'どの映画プロデューサがM1に雇用されて、M2とM3に雇用されて、M4に雇用されて、そして会社に雇用されましたか',
 '何がM0に創設されてそして映画編集者に創設されましたか',
 '何がM2にプロデュースされて、M0に配給されて、M3のプロデューサとエグゼクティブプロデューサにプロデュースされて、そしてM1に配給されましたか',
 '何がM0とM1にプロデュースされてそしてM3の中国の主演にプロデュースされましたか',
 '何がM0の男性の兄弟姉妹にエグゼクティブプロデュースされましたか',
 'M2の前日譚をエグゼクティブプロデュースして、書いて、そして監督した映画監督はM0とM1のどの創設者でしたか',
 '何がM0の撮影監督に書かれてそして編集されましたか',
 'どの撮影監督がM1に影響されましたか',
 '誰がM3、M4、とM5の監督と作家に影響されてそしてM1とM2に影響されましたか',
 'どの映画が衣裳デザイナーに監督されてそしてM1にエグゼクティブプロデュースされましたか',
 '何がM0の監督に編集されてそしてプロデュースされましたか',
 'M3、M4、とM5はM1に書かれてそしてM2に書かれた映画をエグゼクティブプロデュースしてそして書

In [39]:
import nltk 
analyze_sen = "Was a Swedish film producer employed by M1 and employed by M2"
trees = word_parser.parser.parse(analyze_sen.split())
ts = [t for t in trees]
for i in ts:
    i.draw()

In [41]:
trans_sentence

['どの 男性の 人 が M1 と M2 を エグゼクティブ プロデュースしました か']

In [17]:
dic_en2jp['VT']

{'direct': '監督して',
 'produce': 'プロデュースして',
 'edit': '編集して',
 'married': '結婚して',
 'produced': 'プロデュースして',
 'starred': '主演させて',
 'employ': '雇用して',
 'written': '書いて',
 'edited': '編集して',
 'marry': '結婚して',
 'distribute': '配給して',
 'played': '演じて',
 'influenced': '影響して',
 'employed': '雇用して',
 'acquire': '買収して',
 'acquired': '買収して',
 'directed': '監督して',
 'wrote': '書いて',
 'influence': '影響して',
 'found': '創設して',
 'play': '演じて',
 'star': '主演させて',
 'founded': '創設して',
 'write': '書いて',
 'distributed': '配給して'}